# **pyCoreRelator** [![GitHub](https://img.shields.io/badge/GitHub-pyCoreRelator-blue?logo=github)](https://github.com/GeoLarryLai/pyCoreRelator) [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.xxxxxxxx.svg)](https://doi.org/10.5281/zenodo.xxxxxxxx)
## **Workshop Notebook #1: Processing RGB Core Images to Digital Logs**   [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/GeoLarryLai/pyCoreRelator/blob/main/pyCoreRelator_1_RGBimg2log.ipynb)
This notebook demonstrates the general workflow for using modules from **pyCoreRelator** to convert RGB core images into continuous digital color and luminance logs.

### Key Functions from **pyCoreRelator**
- **`rgb_process_and_stitch()`**: Main orchestration function for complete processing workflow
- **`plot_rgbimg_curves()`**: Visualization function creating 3-panel diagnostic plots

For advanced usage, see [FUNCTION_DOCUMENTATION.md](https://github.com/GeoLarryLai/pyCoreRelator/blob/main/FUNCTION_DOCUMENTATION.md) for more details.
<hr>


# **Import Packages**
Load RGB image processing functions from **pyCoreRelator**

In [ ]:
from pyCoreRelator import plot_rgbimg_curves, rgb_process_and_stitch

%matplotlib inline

<hr>

# **Define Core RGB Image Data Structure**

Configure your core segments and processing parameters below. The `rgb_data_reading_structure` dictionary defines:

### Data Reading Structure Format
Image are processed in dictionary order and stitched from top to bottom.

Each segment requires:
- **File name**: Image file name (e.g., `'OSU-M9907-23PC-6-image.bmp'`)
- **Processing parameters**: Dictionary containing:
  - `upper_rgb_threshold`: Upper RGB threshold for filtering bright artifacts
  - `lower_rgb_threshold`: Lower threshold to exclude dark regions
  - `buffer`: Buffer size in pixels around filtered regions
  - `top_trim`: Pixels to remove from top
  - `bottom_trim`: Pixels to remove from bottom
  - `target_luminance`: Sets the mean luminance for normalization (RECOMMENDED: keep consistent across sections).

### Optional Features

For cores with missing segment to be inserted in the stitched image and profile:
- **`empty`**: Use this as a key to insert white space for missing core sections

  **Example with empty section:**
  ```python
  rgb_data_reading_structure = {
      'section-1-image.bmp': {'upper_rgb_threshold': 120, 'buffer': 50, ...},
      'empty': {'rgb_pxlength': 1000, 'rgb_pxwidth': 995},  # <- Missing section
      'section-3-image.bmp': {'upper_rgb_threshold': 120, 'buffer': 50, ...}
  }
  ```


### Core M9907-23PC

In [ ]:
# Basic information for the core to be processed
cruise_name = "M9907"
core_name = "23PC"
core_length_cm = 783  # Adjust this value based on actual core length

# Define critical variables for RGB data reading
rgb_data_reading_structure = {
    f"OSU-{cruise_name}-{core_name}-6-image.bmp": {
        "upper_rgb_threshold": 120,
        "lower_rgb_threshold": 0,
        "buffer": 110,
        "top_trim": 90,
        "bottom_trim": 550,
        "target_luminance": 130
    },
    f"OSU-{cruise_name}-{core_name}-5-image.bmp": {
        "upper_rgb_threshold": 120,
        "lower_rgb_threshold": 0,
        "buffer": 100,
        "top_trim": 80,
        "bottom_trim": 550,
        "target_luminance": 130
    },
    f"OSU-{cruise_name}-{core_name}-4-image.bmp": {
        "upper_rgb_threshold": 120,
        "lower_rgb_threshold": 0,
        "buffer": 130,
        "top_trim": 120,
        "bottom_trim": 530,
        "target_luminance": 130
    },
    f"OSU-{cruise_name}-{core_name}-3-image.bmp": {
        "upper_rgb_threshold": 120,
        "lower_rgb_threshold": 0,
        "buffer": 50,
        "top_trim": 75,
        "bottom_trim": 530,
        "target_luminance": 130
    },
    f"OSU-{cruise_name}-{core_name}-2-image.bmp": {
        "upper_rgb_threshold": 120,
        "lower_rgb_threshold": 0,
        "buffer": 70,
        "top_trim": 35,
        "bottom_trim": 575,
        "target_luminance": 130
    },
    f"OSU-{cruise_name}-{core_name}-1-image.bmp": {
        "upper_rgb_threshold": 120,
        "lower_rgb_threshold": 0,
        "buffer": 80,
        "top_trim": 80,
        "bottom_trim": 500,
        "target_luminance": 130
    }
}


### Core M9907-25PC

In [ ]:
# # Basic information for the core to be processed
# cruise_name = "M9907"
# core_name = "25PC"
# core_length_cm = 797

# # Define critical variables for RGB data reading
# rgb_data_reading_structure = {
#     f"OSU-{cruise_name}-{core_name}-6-image.bmp": {
#         "upper_rgb_threshold": 170,
#         "lower_rgb_threshold": 30,
#         "buffer": 60,
#         "top_trim": 45,
#         "bottom_trim": 120,
#         "target_luminance": 130
#     },
#     f"OSU-{cruise_name}-{core_name}-5-image.bmp": {
#         "upper_rgb_threshold": 170,
#         "lower_rgb_threshold": 60,
#         "buffer": 50,
#         "top_trim": 42,
#         "bottom_trim": 60,
#         "target_luminance": 130
#     },
#     f"OSU-{cruise_name}-{core_name}-4-image.bmp": {
#         "upper_rgb_threshold": 170,
#         "lower_rgb_threshold": 30,
#         "buffer": 50,
#         "top_trim": 95,
#         "bottom_trim": 20,
#         "target_luminance": 130
#     },
#     f"OSU-{cruise_name}-{core_name}-3-image.bmp": {
#         "upper_rgb_threshold": 170,
#         "lower_rgb_threshold": 50,
#         "buffer": 50,
#         "top_trim": 75,
#         "bottom_trim": 65,
#         "target_luminance": 130
#     },
#     f"OSU-{cruise_name}-{core_name}-2-image.bmp": {
#         "upper_rgb_threshold": 170,
#         "lower_rgb_threshold": 30,
#         "buffer": 50,
#         "top_trim": 30,
#         "bottom_trim": 70,
#         "target_luminance": 130
#     },
#     f"OSU-{cruise_name}-{core_name}-1-image.bmp": {
#         "upper_rgb_threshold": 170,
#         "lower_rgb_threshold": 30,
#         "buffer": 50,
#         "top_trim": 30,
#         "bottom_trim": 55,
#         "target_luminance": 130
#     }
# }

### Core M9907-11PC

In [ ]:
# # Basic information for the core to be processed
# cruise_name = "M9907"
# core_name = "11PC"
# core_length_cm = 439

# # Define critical variables for RGB data reading
# rgb_data_reading_structure = {
#     f"OSU-{cruise_name}-{core_name}-3-image.bmp": {
#         "upper_rgb_threshold": 150,
#         "lower_rgb_threshold": 00,
#         "buffer": 70,
#         "top_trim": 80,
#         "bottom_trim": 590,
#         "target_luminance": 130
#     },
#     f"OSU-{cruise_name}-{core_name}-2-image.bmp": {
#         "upper_rgb_threshold": 150,
#         "lower_rgb_threshold": 0,
#         "buffer": 70,
#         "top_trim": 50,
#         "bottom_trim": 540,
#         "target_luminance": 130
#     },
#     f"OSU-{cruise_name}-{core_name}-1-image.bmp": {
#         "upper_rgb_threshold": 150,
#         "lower_rgb_threshold": 0,
#         "buffer": 90,
#         "top_trim": 40,
#         "bottom_trim": 570,
#         "target_luminance": 130
#     }
# }

<hr>

# **Execute the functions:**

## Stitch RGB images and convert to digital log data

**Function: `rgb_process_and_stitch()`**

**What it does:**
1. Loads and processes RGB images from multiple core sections with section-specific parameters
2. Extracts RGB color profiles and luminance values along depth
3. Automatically stitches sections into one complete profile with buffer handling
4. Exports results to image and csv log files

**Key Parameters:**
- `data_reading_structure` *(dict or list)*: Core structure definition with section configurations
- `rgb_data_dir` *(str)*: Base directory path containing RGB image files
- `stitchbuffer` *(int, default=10)*: Number of bin rows to remove at stitching edges to avoid artifacts
- `width_start_pct` *(float, default=0.25)*: Starting percentage of width for analysis strip (crops left side)
- `width_end_pct` *(float, default=0.75)*: Ending percentage of width for analysis strip (crops right side)
- `save_csv` *(bool, default=True)*: Whether to export results to CSV file
- `output_csv` *(str, default=None)*: Full path for output CSV file (required if save_csv=True)
- `total_length_cm` *(float, default=None)*: Total core length in cm for depth conversion (required if save_csv=True)


In [ ]:
all_depths, all_r, all_g, all_b, all_r_std, all_g_std, all_b_std, all_lum, all_lum_std, stitched_image = rgb_process_and_stitch(
    data_reading_structure = rgb_data_reading_structure,
    rgb_data_dir = "example_data/raw_data/Image_data",
    save_csv = True,
    output_csv = f"example_data/processed_data/{cruise_name}-{core_name}/{cruise_name}-{core_name}_RGB.csv",
    total_length_cm = core_length_cm
)

## Visualize the final stitched results

**Function: `plot_rgbimg_curves()`**

**What it does:**
1. Creates a 3-panel figure with:
   - **Left panel:** Core RGB image
   - **Middle panel:** RGB color traces with ±1σ shading  
   - **Right panel:** Standard deviation profile
2. Automatically adjusts figure dimensions based on data aspect ratio
3. Saves composite figures in specified formats (png, jpg, svg, pdf)
4. Creates separate compressed TIFF image if 'tiff' format is requested

**Key Parameters:**
- `depths` *(numpy.ndarray)*: Depth positions in pixels for the RGB data
- `r, g, b` *(numpy.ndarray)*: Red, green, blue color intensity values
- `r_std, g_std, b_std` *(numpy.ndarray)*: Standard deviations of RGB values
- `lum, lum_std` *(numpy.ndarray)*: Relative luminance values and standard deviations
- `img` *(numpy.ndarray)*: Core image array to display
- `core_name` *(str, default="")*: Name of the core for title and filenames
- `save_figs` *(bool, default=False)*: Whether to save figures to files
- `output_dir` *(str, default=None)*: Directory to save figures (required if save_figs=True)
- `fig_format` *(list, default=['png', 'tiff'])*: List of file formats to save (options: 'png', 'jpg', 'svg', 'pdf', 'tiff')
- `dpi` *(int or None, default=150)*: Resolution in dots per inch for saved figures


In [ ]:
plot_rgbimg_curves(all_depths,                          
                    all_r,                               
                    all_g,                               
                    all_b,                               
                    all_r_std,                           
                    all_g_std,                           
                    all_b_std,                           
                    all_lum,                             
                    all_lum_std,                         
                    stitched_image,                      
                    core_name=f"{cruise_name}-{core_name}_RGB",   
                    save_figs = True,
                    output_dir = f"example_data/processed_data/{cruise_name}-{core_name}"
)